In [1]:
pip install transformers

In [2]:
pip install accelerate>=0.20.1

In [3]:
pip install datasets


In [4]:
pip install evaluate

In [5]:
pip install rouge_score

LOAD DATASET

In [6]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json


--2024-02-19 16:14:59--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2248 (2.2K) [text/plain]
Saving to: ‘Dataset.json.4’

Dataset.json.4      100%[===================>]   2.20K  --.-KB/s    in 0s      

2024-02-19 16:14:59 (41.1 MB/s) - ‘Dataset.json.4’ saved [2248/2248]



In [7]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json

--2024-02-19 16:14:59--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2990 (2.9K) [text/plain]
Saving to: ‘Train_Dataset.json.4’

Train_Dataset.json. 100%[===================>]   2.92K  --.-KB/s    in 0s      

2024-02-19 16:14:59 (50.5 MB/s) - ‘Train_Dataset.json.4’ saved [2990/2990]



In [8]:
from datasets import load_dataset



In [9]:
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")


In [10]:
my_dataset_train = load_dataset("json", data_files="Train_Dataset.json", field="documents")


In [11]:
my_dataset_train

DatasetDict({
    train: Dataset({
        features: ['SUMMARIZATION', 'ID', 'TEXT'],
        num_rows: 3
    })
})

In [12]:
my_dataset_test

DatasetDict({
    train: Dataset({
        features: ['SUMMARIZATION', 'ID', 'TEXT'],
        num_rows: 3
    })
})

In [13]:
my_dataset_test['train'][1]

{'SUMMARIZATION': 'Each account and their privileges must periodically review for any changes they may occurred. Also the relative policy must review and improved periodically.',
 'ID': 2,
 'TEXT': 'This policy and its implementation will be subject to internal monitoring and auditing, and the outcomes from these processes will inform and improve practices as part of a commitment to continual improvement. Goldsmiths will also undertake appropriate benchmarking and auditing exercises as may be applicable periodically.'}

Preprocess

In [17]:
from transformers import AutoTokenizer

In [18]:
checkpoint ="facebook/bart-large-cnn"


In [19]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
prefix = "summarize: "
def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["TEXT"]]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True, )
  labels = tokenizer(text_target=examples["SUMMARIZATION"], max_length=512, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [21]:
my_dataset_test = my_dataset_test.map(preprocess_function, batched=True)


In [22]:
my_dataset_train = my_dataset_train.map(preprocess_function, batched=True)

CREATING BATCHING


In [23]:
from transformers import DataCollatorForSeq2Seq


In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer=AutoTokenizer.from_pretrained(checkpoint), model="facebook/bart-large-cnn")

EVALUATE

In [25]:
import evaluate

In [26]:
rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to compute to calculate the ROUGE metric:

In [27]:
model = "facebook/bart-large-cnn"

In [28]:
import numpy as np

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)



    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


Train

In [30]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [31]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [32]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_controls",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate= False,
    fp16=False,
    push_to_hub=False,
)

In [33]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=my_dataset_train["train"],
    eval_dataset=my_dataset_test["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer